<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Pydantic/pydantic_stock_price_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Building an Agentic Stock Price Assistant with Pydantic-AI

## Introduction
This notebook demonstrates how to build an intelligent stock price assistant using Pydantic-AI, an agentic framework that simplifies the creation of AI-powered applications. By combining Pydantic-AI's agent capabilities with Groq's LLama 3.3 model and real-time financial data from YFinance, we create a natural language interface that can understand and respond to queries about stock prices. The integration with Gradio provides an intuitive web interface for user interactions.

## Goals
1. Showcase how to use Pydantic-AI's Agent class to create an intelligent financial assistant
2. Demonstrate tool integration by connecting YFinance for real-time stock data retrieval
3. Implement structured data handling using Pydantic models for type-safe responses
4. Create a conversational interface where users can query stock prices in natural language
5. Build a web-based interface using Gradio for easy interaction with the agent

This implementation highlights the power of agentic frameworks in creating AI applications that can understand user intent, execute specific tools, and return structured responses - all while maintaining a simple and clean architecture.

In [6]:
%pip install -qU pydantic-ai yfinance gradio nest_asyncio

In [2]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [4]:
import nest_asyncio
nest_asyncio.apply()

from pydantic_ai import Agent
from pydantic import BaseModel
import yfinance as yf

class StockPriceResult(BaseModel):
    symbol: str
    price: float
    currency: str = "USD"
    message: str

stock_agent = Agent(
    "groq:llama-3.3-70b-versatile",
    result_type=StockPriceResult,
    system_prompt="You are a helpful financial assistant that can look up stock prices. Use the get_stock_price tool to fetch current data."
)

@stock_agent.tool_plain
def get_stock_price(symbol: str) -> dict:
    ticker = yf.Ticker(symbol)
    price = ticker.fast_info.last_price
    return {
        "price": round(price, 2),
        "currency": "USD"
    }

result = stock_agent.run_sync("What is Apple's current stock price?")
print(f"Stock Price: ${result.data.price:.2f} {result.data.currency}")
print(f"Message: {result.data.message}")

Stock Price: $229.98 USD
Message: The current stock price of Apple (AAPL) is


## User interface in gradio

In [5]:
from pydantic_ai import Agent
from pydantic import BaseModel
import yfinance as yf
import gradio as gr

class StockPriceResult(BaseModel):
    symbol: str
    price: float
    currency: str = "USD"
    message: str

stock_agent = Agent(
    "groq:llama-3.3-70b-versatile",
    result_type=StockPriceResult,
    system_prompt="You are a helpful financial assistant that can look up stock prices. Use the get_stock_price tool to fetch current data."
)

@stock_agent.tool_plain
def get_stock_price(symbol: str) -> dict:
    ticker = yf.Ticker(symbol)
    price = ticker.fast_info.last_price
    return {
        "price": round(price, 2),
        "currency": "USD"
    }

def get_stock_info(query):
    try:
        result = stock_agent.run_sync(query)
        response = f" Stock: {result.data.symbol}\n"
        response += f" Price: ${result.data.price:.2f} {result.data.currency}\n"
        response += f"\n{result.data.message}"
        return response
    except Exception as e:
        return f"Error: {str(e)}"

demo = gr.Interface(
    fn=get_stock_info,
    inputs=gr.Textbox(label="Ask about any stock price", placeholder="What is Apple's current stock price?"),
    outputs=gr.Textbox(label="Stock Information"),
    title="Stock Price AI Assistant",
    description="Ask me about any stock price and I'll fetch the latest information for you!"
)

if __name__ == "__main__":
    demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4ded733d05045feca0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
